In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import CLIPTextModel, CLIPTokenizer

class LearnablePrompt(nn.Module):
    """
    A trainable prompt representation for the Instruct-Pix2Pix model.
    Uses numerical parameters rather than specific tokens.
    """
    def __init__(self, template="Make the image {}", 
                 num_params=10, embedding_dim=768):
        super().__init__()
        self.template = template
        self.tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
        self.text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32")
        
        # Freeze the text encoder - we're only learning the parameters
        for param in self.text_encoder.parameters():
            param.requires_grad = False
            
        # Create trainable continuous parameters
        self.num_params = num_params
        self.embedding_dim = embedding_dim
        self.context_params = nn.Parameter(torch.randn(num_params) * 0.02)
        
        # Linear projection to map from our parameters to the text embedding space
        self.projection = nn.Linear(num_params, embedding_dim)
        
        # Initialize projection with small weights
        nn.init.xavier_uniform_(self.projection.weight, gain=0.0001)
        nn.init.zeros_(self.projection.bias)
        
    def get_params_snapshot(self):
        """Get the current parameter values"""
        return self.context_params.detach().cpu().numpy()
        
    def forward(self, temperature=1.0):
        """
        Generate text prompt using the current parameters
        
        Returns:
            prompt: Text prompt for the diffusion model
            text_embeddings: Text embeddings for advanced use cases
        """
        # Convert parameters to text embeddings through projection
        projected_params = self.projection(self.context_params)
        
        # For the text prompt, we'll use a fixed template with a placeholder
        # The actual impact comes from the embeddings we inject into the diffusion process
        base_prompt = self.template.format("with optimized parameters")
        
        return base_prompt, projected_params
    
    def get_text_embeddings(self, prompt):
        """
        Get text embeddings for a prompt, useful for direct manipulation
        """
        text_inputs = self.tokenizer(
            prompt,
            padding="max_length",
            max_length=self.tokenizer.model_max_length,
            truncation=True,
            return_tensors="pt",
        )
        
        with torch.no_grad():
            text_embeddings = self.text_encoder(
                text_inputs.input_ids.to(self.context_params.device)
            )[0]
            
        return text_embeddings

In [ ]:
import torch
import numpy as np
from PIL import Image
from models.learnable_prompt import LearnablePrompt
from transformers import AutoImageProcessor, AutoModel
from diffusers import StableDiffusionInstructPix2PixPipeline

class AdversarialSetup:
    """
    Complete pipeline for adversarial attack on DINOv2 using Instruct-Pix2Pix.
    Uses numerical parameters for optimization rather than specific tokens.
    """
    def __init__(self, device="cuda" if torch.cuda.is_available() else "cpu"):
        self.device = device
        self.dinov2_model = None
        self.dinov2_processor = None
        self.pix2pix_model = None
        self.prompt_model = None
        
    def load_models(self):
        """Load the required models from HuggingFace"""
        print("Loading DINOv2 model...")
        self.dinov2_processor = AutoImageProcessor.from_pretrained("facebook/dinov2-small")
        self.dinov2_model = AutoModel.from_pretrained("facebook/dinov2-small").to(self.device)
        self.dinov2_model.eval()  # Set to evaluation mode
        
        print("Loading Instruct-Pix2Pix model...")
        self.pix2pix_model = StableDiffusionInstructPix2PixPipeline.from_pretrained(
            "timbrooks/instruct-pix2pix", 
            torch_dtype=torch.float16 if self.device == "cuda" else torch.float32
        ).to(self.device)
        
        # Disable safety checker for this experiment
        self.pix2pix_model.safety_checker = None
        
        # Freeze all Instruct-Pix2Pix parameters
        for param in self.pix2pix_model.parameters():
            param.requires_grad = False
            
        print("Models loaded successfully!")
    
    def initialize_prompt_model(self, template="Transform the image to {}", 
                              num_params=10, embedding_dim=768):
        """
        Initialize the learnable prompt model with numerical parameters
        
        Args:
            template: Template string for the prompt
            num_params: Number of trainable parameters
            embedding_dim: Dimension of the embedding space
        """
        self.prompt_model = LearnablePrompt(
            template=template,
            num_params=num_params,
            embedding_dim=embedding_dim
        ).to(self.device)
        
        return self.prompt_model
    
    def get_dinov2_representation(self, image):
        """
        Get DINOv2 representation for an image
        image: PIL Image or preprocessed tensor
        """
        if isinstance(image, Image.Image):
            # Preprocess the image
            inputs = self.dinov2_processor(images=image, return_tensors="pt").to(self.device)
            with torch.no_grad():
                outputs = self.dinov2_model(**inputs)
            return outputs.last_hidden_state[:, 0]  # CLS token embedding
        else:
            # Assume image is already a preprocessed tensor
            with torch.no_grad():
                outputs = self.dinov2_model(pixel_values=image)
            return outputs.last_hidden_state[:, 0]
            
    def apply_pix2pix(self, image, prompt_text, prompt_embedding=None, num_steps=20):
        """
        Apply Instruct-Pix2Pix to generate a modified image
        
        Args:
            image: Input image
            prompt_text: Text prompt
            prompt_embedding: Optional embedding to influence generation
            num_steps: Number of diffusion steps
        """
        # For this example, we'll use the standard pipeline interface
        # In a more advanced implementation, you could modify the pipeline internals
        # to directly inject the embeddings
        output = self.pix2pix_model(
            prompt_text,
            image=image, 
            num_inference_steps=num_steps, 
            image_guidance_scale=1.5,
            guidance_scale=7.0
        )
        return output.images[0]
        
    def representation_distance(self, rep1, rep2, metric="cosine"):
        """
        Calculate distance between two representations
        We want to maximize this distance (minimize similarity)
        """
        if metric == "cosine":
            # Normalize vectors
            rep1_norm = rep1 / rep1.norm(dim=1, keepdim=True)
            rep2_norm = rep2 / rep2.norm(dim=1, keepdim=True)
            # Compute cosine distance (1 - similarity)
            similarity = (rep1_norm * rep2_norm).sum(dim=1)
            distance = 1 - similarity
        elif metric == "l2":
            # L2 distance
            distance = torch.norm(rep1 - rep2, dim=1)
        else:
            raise ValueError(f"Unknown distance metric: {metric}")
            
        return distance.mean()
    
    def full_pipeline(self, image, get_gradients=False):
        """
        Run the full adversarial pipeline with numerical parameters:
        1. Generate prompt from learnable parameters
        2. Apply Pix2Pix to modify image
        3. Get DINOv2 representations of original and modified images
        4. Calculate representation distance
        """
        # Step 1: Generate prompt from learnable parameters
        prompt_text, prompt_embedding = self.prompt_model()
        
        # Step 2: Apply Pix2Pix to modify image
        modified_image = self.apply_pix2pix(
            image, 
            prompt_text=prompt_text, 
            prompt_embedding=prompt_embedding
        )
        
        # Step 3: Get DINOv2 representations
        # For original image (no gradients needed)
        with torch.no_grad():
            original_inputs = self.dinov2_processor(images=image, return_tensors="pt").to(self.device)
            original_outputs = self.dinov2_model(**original_inputs)
            original_rep = original_outputs.last_hidden_state[:, 0]  # CLS token 
        
        # For modified image (with gradients if needed)
        if get_gradients:
            # Convert PIL to tensor with preprocessing
            modified_inputs = self.dinov2_processor(images=modified_image, return_tensors="pt").to(self.device)
            modified_outputs = self.dinov2_model(**modified_inputs)
            modified_rep = modified_outputs.last_hidden_state[:, 0]  # CLS token
        else:
            with torch.no_grad():
                modified_inputs = self.dinov2_processor(images=modified_image, return_tensors="pt").to(self.device)
                modified_outputs = self.dinov2_model(**modified_inputs)
                modified_rep = modified_outputs.last_hidden_state[:, 0]  # CLS token
        
        # Step 4: Calculate representation distance
        distance = self.representation_distance(original_rep, modified_rep)
        
        return modified_image, original_rep, modified_rep, distance, prompt_text

In [ ]:
import torch
import torch.optim as optim
import numpy as np
from tqdm.auto import tqdm
from models.adversarial_setup import AdversarialSetup
from PIL import Image

def train_adversarial_prompt(adv_setup, original_image, num_iterations=50, 
                           lr=0.01):
    """
    Train the prompt model with numerical parameters to maximize representation distance
    
    Args:
        adv_setup: Initialized AdversarialSetup object
        original_image: PIL Image to start from
        num_iterations: Number of training iterations
        lr: Learning rate
        
    Returns:
        history: Dictionary containing training history
    """
    optimizer = optim.Adam(adv_setup.prompt_model.parameters(), lr=lr)
    
    history = {
        'distance': [],
        'prompts': [],
        'images': [],
        'parameter_values': []
    }
    
    # Main training loop
    for i in tqdm(range(num_iterations)):
        optimizer.zero_grad()
        
        # Apply full pipeline with gradients
        modified_image, original_rep, modified_rep, distance, prompt = adv_setup.full_pipeline(
            original_image, get_gradients=True
        )
        
        # Loss is negative distance (we want to maximize distance)
        loss = -distance
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        
        # Log progress
        history['distance'].append(distance.item())
        history['prompts'].append(prompt)
        
        # Get parameter values for analysis
        param_values = adv_setup.prompt_model.get_params_snapshot()
        history['parameter_values'].append(param_values)
        
        # Save image every 10 iterations
        if i % 10 == 0 or i == num_iterations - 1:
            history['images'].append(modified_image)
            
        # Print progress
        if i % 5 == 0:
            print(f"Iteration {i}, Distance: {distance.item():.4f}, Prompt: {prompt}")
            print(f"Parameter values range: [{param_values.min():.4f}, {param_values.max():.4f}]")
    
    return history

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def visualize_training_results(original_image, history):
    """Visualize the training progress and results with numerical parameters"""
    # Plot distance over iterations
    plt.figure(figsize=(10, 5))
    plt.plot(history['distance'])
    plt.title('Representation Distance Over Iterations')
    plt.xlabel('Iteration')
    plt.ylabel('Distance')
    plt.grid(True)
    plt.show()
    
    # Display original and modified images
    num_images = len(history['images'])
    fig, axes = plt.subplots(1, num_images + 1, figsize=(5*(num_images+1), 5))
    
    # Show original image
    axes[0].imshow(original_image)
    axes[0].set_title("Original")
    axes[0].axis('off')
    
    # Show modified images at different iterations
    for i, img in enumerate(history['images']):
        iter_num = i * 10 if i < num_images - 1 else len(history['distance']) - 1
        axes[i+1].imshow(img)
        axes[i+1].set_title(f"Iteration {iter_num}")
        axes[i+1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Plot parameter evolution
    param_values = np.array(history['parameter_values'])
    plt.figure(figsize=(12, 6))
    
    # Plot each parameter's value over time
    for i in range(param_values.shape[1]):
        plt.plot(param_values[:, i], label=f'Param {i}')
    
    plt.title('Parameter Values Over Training')
    plt.xlabel('Iteration')
    plt.ylabel('Value')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    # Show heatmap of parameter values
    plt.figure(figsize=(12, 6))
    plt.imshow(param_values.T, aspect='auto', cmap='viridis')
    plt.colorbar(label='Parameter Value')
    plt.title('Parameter Evolution Heatmap')
    plt.xlabel('Iteration')
    plt.ylabel('Parameter Index')
    plt.tight_layout()
    plt.show()

In [ ]:
from models.adversarial_setup import AdversarialSetup
from train import train_adversarial_prompt
from utils.visualization import visualize_training_results
from PIL import Image

def main():
    # Load image
    original_image = Image.open("path/to/your/image.jpg").convert("RGB")
    
    # Setup with numerical parameters
    adv_setup = AdversarialSetup()
    adv_setup.load_models()
    
    # Initialize with 10 trainable numerical parameters
    adv_setup.initialize_prompt_model(
        template="Transform the image to {}",
        num_params=10,
        embedding_dim=768
    )
    
    # Train
    history = train_adversarial_prompt(
        adv_setup, 
        original_image,
        num_iterations=50,
        lr=0.01
    )
    
    # Visualize results
    visualize_training_results(original_image, history)
    
    # Save final adversarial image
    history['images'][-1].save("adversarial_result.jpg")
    print(f"Final adversarial image saved to adversarial_result.jpg")
    
    return history

if __name__ == "__main__":
    main()